In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

class StopExecution(Exception):
    def _render_traceback_(self):
        return []



In [ ]:
import sys
sys.path.append("..")
import pickle as pkl
import hist
from analyzer.datasets import SampleManager
from analyzer.core import AnalysisResult
import math
import torch
import gpytorch
from torch.masked import masked_tensor, as_masked_tensor
import numpy as np
import matplotlib.pyplot as plt
from analyzer.plotting.plots_1d import drawAs1DHist
from analyzer.plotting.plots_2d import drawAs2DHist, addTitles2D
from analyzer.plotting.plottables import PlotObject
from analyzer.plotting.mplstyles import loadStyles
import analyzer.plotting as plot
from fitting import regression
from fitting import plot_tools as fplt
loadStyles()
torch.set_default_dtype(torch.float64)

In [ ]:
res = AnalysisResult.fromFile("../results/data_control.pkl")

sample_manager = SampleManager()
sample_manager.loadSamplesFromDirectory("../datasets")

res.results["CR0b_Data2018"].histograms["h_njet"] 

hists = res.getMergedHistograms(sample_manager)

In [ ]:
bkg_name="CR0b_Data2018"
complete_hist = hists["m14_vs_m24"]
narrowed = complete_hist
#orig =  complete_hist[...,hist.loc(1100):hist.loc(2500),hist.loc(400):hist.loc(2000)]
#narrowed =  complete_hist[...,hist.loc(0):hist.loc(3000),hist.loc(0):hist.loc(1)]
#narrowed =  complete_hist[...,hist.loc(1150):hist.loc(3000),hist.loc(0.4):hist.loc(1)]
#orig = orig / orig.values().max()
#narrowed =  complete_hist[...,hist.loc(1150):hist.loc(3000),hist.loc(400):hist.loc(3000)]
narrowed =  complete_hist[...,hist.loc(0):hist.loc(3000),hist.loc(0):hist.loc(3000)]
#narrowed = orig
#narrowed = orig[...,::hist.rebin(15),::hist.rebin(12)]
narrowed = narrowed[...,::hist.rebin(1),::hist.rebin(1)]

qcd_hist = narrowed[bkg_name,...]
#orig_qcd_hist = orig[bkg_name,...]

#sig_hist = narrowed["signal_312_1500_900",...]

qcd_hist = narrowed[bkg_name,...] 


In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
def addColorbar(ax, vals):
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = plt.colorbar(vals, cax=cax)
    cax.get_yaxis().set_offset_position("left")
    ax.cax = cax

In [ ]:
fig,ax= plt.subplots(1,2,figsize=(10,3))
_ = drawAs2DHist(ax[0], PlotObject.fromHist(qcd_hist))
#_ = drawAs2DHist(ax[1], PlotObject.fromHist(sig_hist))

In [ ]:
import fitting.regression as fr
window = [(1300, 1750), (400, 700)]
rebin = 1
mask_f = fr.rectMasker(window)
mask_f = fr.ellipseMasker(torch.tensor([1500.0,400.0]), 150.0,100.0)
train_data = regression.makeRegressionData(qcd_hist[hist.rebin(rebin),hist.rebin(rebin)], mask_f, exclude_less=0.001)
test_data = regression.makeRegressionData(qcd_hist[hist.rebin(rebin),hist.rebin(rebin)], None, exclude_less=0.001)
train_transform = regression.getNormalizationTransform(train_data)
test_transform = regression.getNormalizationTransform(test_data)
normalized_train_data = train_transform.transform(train_data)
normalized_test_data = test_transform.transform(test_data)
fig,ax = plt.subplots(2,2,figsize=(10,10),layout="tight")
fplt.simpleGrid(ax[0][0], train_data.E, train_data.X, train_data.Y)
fplt.simpleGrid(ax[0][1], test_data.E, test_data.X, test_data.Y)
fplt.simpleGrid(ax[1][0], normalized_train_data.E, normalized_train_data.X, normalized_train_data.Y)
fplt.simpleGrid(ax[1][1], normalized_test_data.E, normalized_test_data.X, normalized_test_data.Y)

In [ ]:
class InducingPointModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, inducing):
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.base_covar_module = grbf
        self.covar_module = gpytorch.kernels.InducingPointKernel(self.base_covar_module, inducing_points=inducing.clone(), likelihood=likelihood)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


In [ ]:
from fitting import regression
import fitting
from gpytorch.kernels import ScaleKernel as SK
from gpytorch.kernels import RBFKernel as RBF
import fitting.models as models
if torch.cuda.is_available():   
    print("Loading train to gpu")
    train = regression.DataValues(normalized_train_data.X.cuda(), normalized_train_data.Y.cuda(), normalized_train_data.V.cuda(), None)
    print("Loaded train to gpu")
else:
    train = normalized_train_data

linear=gpytorch.kernels.LinearKernel(ard_num_dims=2,num_dimensions=2)
rq=SK(gpytorch.kernels.RQKernel(ard_num_dims=2))

smk = gpytorch.kernels.SpectralMixtureKernel(ard_num_dims=2, num_mixtures=4)
gsmk=models.GeneralSpectralMixture(num_mixtures=1, ard_num_dims=2)
peak_smk=models.PeakedSMK(ard_num_dims=2, num_mixtures=16)
smk.initialize_from_data(train.X, train.Y)
peak_smk.initialize_from_data(train.X, train.Y)

peaked_rbf=SK(models.PeakedRBF(ard_num_dims=2))
peaked_grbf=SK(models.PeakedGRBF(ard_num_dims=2))

NNRBF = fitting.models.wrapNN("NNRBFKernel", gpytorch.kernels.RBFKernel)

nnrbf = SK(NNRBF(feature_dim=2))
grq=SK(models.GeneralRQ(ard_num_dims=2))
grbf = SK(models.GeneralRBF(ard_num_dims=2))
gmatern = SK(models.GeneralMatern(ard_num_dims=2))
poly = SK(gpytorch.kernels.PiecewisePolynomialKernel(ard_num_dims=2))
lk = SK(gpytorch.kernels.LinearKernel(ard_num_dims=2,num_dimensions=2))
rbf = SK(gpytorch.kernels.RBFKernel(ard_num_dims=2))
matern = SK(gpytorch.kernels.MaternKernel(ard_num_dims=2))

grid_size = gpytorch.utils.grid.choose_grid_size(train.X,0.3)
grid_matern = SK(gpytorch.kernels.GridInterpolationKernel(gpytorch.kernels.RBFKernel(ard_num_dims=2),grid_size=grid_size,num_dims=2))



model,likelihood = regression.createModel(train, kernel=grid_matern)

if torch.cuda.is_available():   
    print("Loading model to gpu")
    model = model.cuda()
    likelihood = likelihood.cuda()
    #train = regression.DataValues(train.inputs.cuda(), train.outputs.cuda(), train.variances.cuda(), None)
    print("Loaded model to gpu")
    
print(model)
model,likelihood = regression.optimizeHyperparams(model,likelihood, train, bar=False, iterations=100, lr=0.05)
print("DONE")

In [ ]:
params = list(model.named_parameters_and_constraints())
for name,param,constraint in params:
    if constraint:
        real_param = constraint.transform(param)
    else:
        real_param = param
    print(f"{name.replace('raw_',''):{max(len(x[0]) for x in params)+4}} {real_param.detach().numpy().round(3)}" )


In [ ]:
import linear_operator
#with linear_operator.settings.max_cg_iterations(10000):
#    pred = regression.getPrediction(model, likelihood, test_data)
pred = regression.getPrediction(model, likelihood, normalized_test_data)


from mpl_toolkits.axes_grid1 import make_axes_locatable
import itertools as it
from fitting.plot_tools import simpleGrid
#window = [(1300,1550),(400,600)]
print(test_transform)
pred = test_transform.iTransform(
    regression.DataValues(normalized_test_data.X, pred.mean, pred.variance, normalized_test_data.E))


if window:
    mask = regression.getBlindedMask(pred.X, pred.Y, test_data.Y, test_data.V, window)
    bpred_mean = pred.Y[mask]
    obs_mean = test_data.Y[mask]
    obs_var = test_data.V[mask]
    chi2 = torch.sum((obs_mean - bpred_mean)**2 / obs_var) / torch.count_nonzero(mask)
    avg_pull = torch.sum(torch.abs((obs_mean - bpred_mean)) / torch.sqrt(obs_var)) / torch.count_nonzero(mask)
    print(f"Chi^2/bins = {chi2}")
    print(f"Avg Abs pull = {avg_pull}")

all_pulls = (pred.Y - test_data.Y) / torch.sqrt(test_data.V)
all_x2 = (pred.Y - test_data.Y)**2 / test_data.V
x2 = torch.sum(all_x2)

def addColorbar(ax, vals):
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = plt.colorbar(vals, cax=cax)
    cax.get_yaxis().set_offset_position("left")
    ax.cax = cax

fig, ax=plt.subplots(layout="tight")
f = simpleGrid(ax, train_data.E, train_data.X, train_data.Y)
ax.set_title("Masked Inputs (Training)")
addTitles2D(ax, PlotObject.fromHist(qcd_hist))
#fig.savefig(figpath / "training_points.pdf")

fig, ax=plt.subplots(figsize=(10,10), layout="tight")
f = simpleGrid(ax, pred.E, pred.X, pred.Y)
ax.set_title("GPR Mean Prediction")
addTitles2D(ax, PlotObject.fromHist(qcd_hist))
#fig.savefig(figpath / "gpr_mean.pdf")


fig, ax=plt.subplots(figsize=(10,10), layout="tight")
f = simpleGrid(ax, test_data.E, test_data.X, test_data.Y)
ax.set_title("Observed Outputs")
#fig.savefig(figpath / "observed_outputs.pdf")



#drawAs2DHist(ax[1,1], PlotObject.fromHist(sig_hist))
#addTitles2D(ax[1,1], PlotObject.fromHist(sig_hist))
#ax[1,1].set_title("Signal MC")

fig, ax=plt.subplots(figsize=(10,10), layout="tight")
f = simpleGrid(ax, test_data.E, test_data.X, test_data.V)
ax.set_title("Observed Variances")
addTitles2D(ax, PlotObject.fromHist(qcd_hist))

#fig.savefig(figpath / "observed_variances.pdf")


fig, ax=plt.subplots(figsize=(10,10), layout="tight")
f = simpleGrid(ax, pred.E, pred.X, pred.V)
ax.set_title("Pred Variances")
addTitles2D(ax, PlotObject.fromHist(qcd_hist))

#fig.savefig(figpath / "predicted_variances.pdf")


fig, ax=plt.subplots(figsize=(10,10), layout="tight")
f = simpleGrid(ax, test_data.E, test_data.X, (test_data.Y - pred.Y)/ torch.sqrt(test_data.V))
f.set_clim(-5,5)
ax.set_title("Pulls")
addTitles2D(ax, PlotObject.fromHist(qcd_hist))
ax.cax.set_ylabel(r"$\frac{N_{obs}-N_{pred}}{\sigma_{p}}$")
#fig.savefig(figpath / "pulls.pdf")



import uhi
fig, ax=plt.subplots(figsize=(10,10), layout="tight")
p = PlotObject.fromHist(uhi.numpy_plottable.ensure_plottable_histogram(np.histogram(all_pulls[torch.abs(all_pulls) < np.inf], bins=20)))
drawAs1DHist(ax, p, yerr=False)
ax.set_xlabel(r"$\frac{N_{obs}-N_{pred}}{\sigma_{o}}$")
ax.set_ylabel("Count")
#fig.savefig(figpath / "pulls_hist.pdf")

In [ ]:
import fitting.plot_tools as fpt
import fitting
import analyzer.plotting as plotting
from pathlib import Path
import importlib

importlib.reload(fpt)
dim = 1

pred_mean,_ = fitting.regression.pointsToGrid(test_data.X, pred.Y, test_data.E)
pred_var,_ = fitting.regression.pointsToGrid(test_data.X, pred.V, test_data.E)
obs_vals,_ = fitting.regression.pointsToGrid(test_data.X, test_data.Y, test_data.E)
obs_vars, filled = fitting.regression.pointsToGrid(test_data.X, test_data.V, test_data.E)

#(figpath /"slices" / f"along_{dim}").mkdir(parents=True, exist_ok=True)
for val, f, ax in fpt.createSlices(
    pred_mean.hist,
    pred_var.hist,
    obs_vals.hist,
    obs_vars.hist,
    test_data.E,
    filled,
    observed_title="CRData", window_2d=window, dim=dim):
    plotting.addTitles1D(ax, plotting.PlotObject.fromHist(qcd_hist[:,sum]))
    #f.savefig(figpath /"slices" / f"along_{dim}" /  (f"slice_{round(float(val),3)}".replace(".","p") + ".pdf"))
    #plt.close(f)
    
    